## TP1 VpC2

In [1]:
import numpy as np

In [2]:
# Definimos las funciones convolve y zero_padding

def convolve(X, Kernel, Bias):
  convolucion = np.sum(X * Kernel) + float(Bias)
  return convolucion


def zero_padding(X, pad):
    pading = np.pad(X, (pad, pad), mode='constant', constant_values = (0,0))
    return pading

In [3]:
#Defino la clase para R2
class conv():
    def __init__(self, kernel_num, kernel_size , padding, stride ):
        self.kernel_num = kernel_num
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        
    # Método forward:
    
    def forward(self, input, kernel, bias):  
        (ncapas, filter_size, filter_size) = kernel.shape
        (nH, nW) = input.shape
        print("cantidad de capas = ", ncapas)
        #if ncapas != self.kernel_num:
            #print('No coincide el número de filtros: {} con el declarado en la instancia: {}'. format(ncapas, self.kernel_num))
            #return
        if self.padding == 'VALID':
            p = 0
        elif self.padding == 'SAME':
            p = int(((self.stride - 1) * nH - self.stride + filter_size) / 2)
        else:
            p = self.padding

        # calculo la dimencion de salida:
        out_H = int((nH + 2 * p - filter_size)/self.stride + 1)
        out_W = int((nW + 2 * p - filter_size)/self.stride + 1)
       
        ini = np.zeros([ncapas, out_H, out_W])

        # Padding

        Layer_Padded = zero_padding(input, p)
        
        for j in range(out_H):
            y_inicio = self.stride * j
            y_fin = y_inicio + filter_size

            for i in range(out_W):
                x_inicio = self.stride * i
                x_fin = x_inicio + filter_size
                Slice = Layer_Padded[y_inicio:y_fin, x_inicio:x_fin]
                
                for capa in range(ncapas):
                    filt_ = kernel[capa, :, :]     
                    ini[capa, j, i]= np.float32(convolve(Slice, filt_, bias))        
  
        return ini

    # Backward

    def backw(self, input, kernel, dL_dO):
        (nH, nW) = input.shape  
        (ncapas, filter_size, filter_size) = kernel.shape

        if self.padding == 'VALID':
            p = 0

        elif self.padding == 'SAME':
            p = int(((self.stride - 1) * nH - self.stride + filter_size) / 2)
       
        else:
            p = self.padding
        # calculo la dimencion de salida:
        out_H = int((nH + 2 * p - filter_size) / self.stride + 1)
        out_W = int((nW + 2 * p - filter_size) / self.stride + 1)

        dL_dF = np.zeros([ncapas, out_H, out_W])
        
        Layer_Padded = zero_padding(input, p)
        
        for j in range(out_H):
            y_inicio = self.stride * j
            y_fin = y_inicio + filter_size
            for i in range(out_W):
                x_inicio = self.stride * i
                x_fin = x_inicio + filter_size
                
                layer = Layer_Padded[y_inicio:y_fin, x_inicio:x_fin]
                
                for c in range(ncapas):
                    dLdO_ = dL_dO[c, :, :]     
                    dL_dF[c, j, i]= np.float32(convolve(layer, dL_dO, 0)) 
        
        # Roto la matriz
        dL_dX = np.zeros([ncapas, out_H, out_W])
        for c in range(ncapas):
            filt_ = kernel[c, :, :]
            filt_ = np.rot90(filt_, k = 2, axes = (0, 1))
            dL_dX[c, :, :] = np.float32(convolve(filt_, dL_dO, 0)) 
            
        return dL_dX, dL_dF
    

In [4]:
array = np.random.randint(0,10,(4,4))
array

array([[4, 9, 1, 7],
       [0, 3, 2, 6],
       [3, 4, 3, 3],
       [0, 6, 7, 5]])

In [5]:
model = conv(kernel_num = 2, kernel_size = 3, padding = 0, stride = 1)
k = np.random.randint(0,10,(2,3,3))
dL_dO = np.random.randint(0,10,(2,3,3))
k

array([[[0, 3, 1],
        [8, 9, 4],
        [1, 4, 4]],

       [[5, 1, 6],
        [6, 0, 6],
        [0, 0, 5]]])

In [6]:
Q = model.forward(array, k, 1)

cantidad de capas =  2


In [7]:
dLdX, dLdF = model.backw(array, k, dL_dO)
dLdF

array([[[238., 251.],
        [259., 283.]],

       [[238., 251.],
        [259., 283.]]])

In [8]:
dLdX

array([[[237., 237.],
        [237., 237.]],

       [[236., 236.],
        [236., 236.]]])